# Data Consolidattion 

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
#!pip install geopandas
#!pip install geopy
from geopy.geocoders import Nominatim
import re

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [3]:
filenames_2019 = all_filenames[:6]

filenames_2020_2021 = all_filenames[6:]

In [4]:
#combine all files in the list
df_19 = pd.concat([pd.read_csv(f) for f in filenames_2019 ])

df_20_21 = pd.concat([pd.read_csv(f) for f in filenames_2020_2021 ])

In [5]:
# Create 4 empty columns in 2019 data

df_19['start_lat'] = " "
df_19['start_lng'] = " "
df_19['end_lat'] = " "
df_19['end_lng'] = " "

df_19['ride_id'] = " "
df_19['rideable_type'] = " "


In [6]:
# Create functions for geocoding 

def extract_lat(address):
    try:
        location = geolocator.geocode(address)
        return location.latitude
    except:
        return ''

    
def extract_long(address):
    try:
        location = geolocator.geocode(address)
        return location.longitude
    except:
        return ''

In [7]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="example app")

### Make the same columns for all datasets 

In [24]:
df_19 = df_19.loc[:, ["ride_id","rideable_type","Start date","End date", "Start station", "Start station number"
           ,"End station", "End station number",'start_lat', 'start_lng', 'end_lat', 'end_lng','Member type','Bike number','Duration']]


df_19.columns = ['ride_id', 'rideable_type', 'started_at', 'ended_at','start_station_name', 'start_station_id', 
                 'end_station_name','end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng','member_casual', 'Bike number', 'Duration']


In [26]:
# Add columns in 2020_2021 data 

df_20_21['Bike number']= ""

#convert columns to datetime
df_20_21[['started_at','ended_at']] = df_20_21[['started_at','ended_at']].apply(pd.to_datetime)

#calculate difference between dates
df_20_21['Duration'] = (df_20_21['ended_at'] - df_20_21['started_at']) / np.timedelta64(1, 's')


In [41]:
# Change the wording in order to be aligned

df_19["member_casual"] = df_19["member_casual"].map(lambda x: "member" if x=="Member" else "casual")

### Consolidate all data 

In [43]:
df = pd.concat([df_19, df_20_21])

In [69]:
# Add semi annual column 

df[['started_at','ended_at']] = df[['started_at','ended_at']].apply(pd.to_datetime)

df['semi_annual'] = df['started_at'].map(lambda x:"H2 2019" if x.year==2019 else ("H2 2020" if x.year==2020 else "H2 2021"))


In [71]:
#export to csv
df.to_csv( "data.csv", index=False)

### Import data 

In [2]:
df = pd.read_csv("data.csv")

C:\Users\Tsaro\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (8,9,10,11,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### EDA 

In [74]:
df.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id      float64
end_station_name       object
end_station_id        float64
start_lat              object
start_lng              object
end_lat                object
end_lng                object
member_casual          object
Bike number            object
Duration              float64
semi_annual            object
dtype: object

In [75]:
df.shape

(4695622, 16)

In [76]:
df.describe()

,start_station_id,end_station_id,Duration
count,4.516959e+06,4.493775e+06,4.695622e+06
mean,3.135274e+04,3.135348e+04,1.374490e+03
std,4.148156e+02,4.117560e+02,4.522095e+04
min,0.000000e+00,0.000000e+00,-1.743833e+06
25%,3.120500e+04,3.121200e+04,4.210000e+02
50%,3.127100e+04,3.126900e+04,7.340000e+02
75%,3.160300e+04,3.160300e+04,1.266000e+03
max,3.290200e+04,3.290200e+04,1.442492e+07


In [88]:
# Missing values
df.isnull().sum()

ride_id                     6
rideable_type               0
started_at                  0
ended_at                    0
start_station_name     178663
start_station_id       178663
end_station_name       201847
end_station_id         201847
start_lat                   8
start_lng                   8
end_lat                  7553
end_lng                  7553
member_casual               0
Bike number           2895766
Duration                    0
semi_annual                 0
dtype: int64

In [3]:
np.count_nonzero(df.start_station_id.unique())

683

In [4]:
np.count_nonzero(df.end_station_id.unique())

683

In [5]:
np.count_nonzero(df.start_station_name.dropna().unique())

713

In [6]:
np.count_nonzero(df.end_station_name.dropna().unique())

713

In [7]:
start = df.start_station_name.dropna().unique()
end = df.end_station_name.dropna().unique()

In [8]:
from opencage.geocoder import OpenCageGeocode

def geo_lat(x):
    key = "18f60230b17a4b76a633dd7eb0f9ee98"
    geocoder = OpenCageGeocode(key)
    address = x
    result = geocoder.geocode(address, no_annotations="1")  
    
    return result[0]['geometry']['lat']

def geo_lng(x):
    key = "18f60230b17a4b76a633dd7eb0f9ee98"
    geocoder = OpenCageGeocode(key)
    address = x
    result = geocoder.geocode(address, no_annotations="1")  
    
    return result[0]['geometry']['lng']

In [9]:
addresses = start
key = "18f60230b17a4b76a633dd7eb0f9ee98"
geocoder = OpenCageGeocode(key)
latitudes_start = []
longitudes_start = []
for address in addresses: 
    result = geocoder.geocode(address, no_annotations="1")  
    
    if result and len(result):  
        longitude = result[0]["geometry"]["lng"]  
        latitude = result[0]["geometry"]["lat"] 
    else:  
        longitude = "N/A"  
        latitude = "N/A"  
    
    latitudes_start.append(latitude) 
    longitudes_start.append(longitude)

In [10]:
addresses = end
key = "18f60230b17a4b76a633dd7eb0f9ee98"
geocoder = OpenCageGeocode(key)
latitudes_end = []
longitudes_end = []
for address in addresses: 
    result = geocoder.geocode(address, no_annotations="1")  
    
    if result and len(result):  
        longitude = result[0]["geometry"]["lng"]  
        latitude = result[0]["geometry"]["lat"] 
    else:  
        longitude = "N/A"  
        latitude = "N/A"  
    
    latitudes_end.append(latitude) 
    longitudes_end.append(longitude)

In [11]:
df_geo = pd.DataFrame()

In [12]:
df_geo['start'] = start

In [13]:
df_geo['end'] = end

In [15]:
df_geo['start_lat'] = latitudes_start

df_geo['start_long'] = longitudes_start

df_geo['end_lat'] = latitudes_end

df_geo['end_long'] = longitudes_end


In [21]:
#export to csv
df_geo.to_csv( "geo.csv", index=False)

In [26]:
# change column names 
df_geo.columns = ['start_station_name',"end_station_name", 'start_lat', 'start_lng','end_lat','end_lng']


In [42]:
df_geo_start = df_geo[['start_station_name', 'start_lat', 'start_lng']]

df_geo_end = df_geo[['end_station_name', 'end_lat', 'end_lng']]

In [34]:
df_19 = df[df.semi_annual=="H2 2019"]

In [47]:
# Connect geocoding to dataframe of 2019 
n_df_19 = pd.merge(df_19, df_geo_start, on ='start_station_name', how ='left')

df_19_f = pd.merge(n_df_19, df_geo_end, on ='end_station_name', how ='left')

In [49]:
df_19_f.drop(['start_lat_x', "start_lng_x", 'end_lat_x','end_lng_x'],axis=1, inplace=True)

In [53]:
# change column names 
df_19_f.columns = ['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'member_casual', 'Bike number', 'Duration',
       'semi_annual', 'start_lat', 'start_lng', 'end_lat', 'end_lng']


# change the column order  

df_19_f = df_19_f[['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'Bike number', 'Duration', 'semi_annual']]


In [ ]:
df_rest = df[df.semi_annual!="H2 2019"]

df_rest.reset_index(drop=True, inplace=True)

df_vf = pd.merge(df_19_f,df_rest, how='outer')


In [87]:
#export to csv
df_vf.to_csv( "final_data.csv", index=False)
